# 4.4.4. Regresión polinomial
Ahora podemos explorar estos conceptos de forma interactiva ajustando polinomios a los datos.

In [132]:
use strict;
use warnings;
use Data::Dump qw(dump);
use Data::Dumper qw(Dumper);
use AI::MXNet::Gluon qw(gluon);
use AI::MXNet qw(mx);
use List::Util qw(max min shuffle);
#use PDL qw(pdl);
use d2l;
use d2l::Timer;
use d2l::Animator;
use d2l::Accumulator;
IPerl->load_plugin('Chart::Plotly');
use Chart::Plotly::Plot;


## 4.4.4.1. Generación del conjunto de datos
Primero necesitamos datos. Dado x , usaremos el siguiente polinomio cúbico para generar las etiquetas en los datos de entrenamiento y prueba:

(4.4.2)
y=5+1.2x−3.4x22!+5.6x33!+ϵ donde ϵ∼N(0,0.12).
 
El término de ruido ϵ sigue una distribución normal con una media de 0 y una desviación estándar de 0,1. Para la optimización, normalmente queremos evitar valores muy grandes de gradientes o pérdidas. ¡Esta es la razón por la cual las características se reescalan de xi a xii! . Nos permite evitar valores muy grandes para exponentes i grandes. Sintetizaremos 100 muestras cada una para el conjunto de entrenamiento y el conjunto de prueba.

In [133]:
my $max_degree = 20;  # Maximum degree of the polynomialmy 
my $n_train= 100;
my $n_test = 100;  # Training and test dataset sizes
my $true_w = mx->nd->zeros([$max_degree]);  # Allocate lots of empty space

$true_w->[ 0..4 ] = mx->nd->array([5, 1.2, -3.4, 5.6]);

my $n_blast = $n_train+ $n_test;
my $size = mx->nd->array([$n_blast,1]);

$features = mx->nd->random_normal(0, 0.01, $size->asarray);
my $out = shuffle($features);

my $par = mx->nd->arange(stop => $max_degree);
my $param = $par->reshape([1, -1]);

my $poly_features = $features ** $param;

for my $i (0..($max_degree -1)){
    
}

Nuevamente, los monomios almacenados en poly_features son reescalados por la función gamma, donde Γ(n)=(n−1)! . Eche un vistazo a las primeras 2 muestras del conjunto de datos generado. El valor 1 es técnicamente una característica, es decir, la característica constante correspondiente al sesgo.

In [134]:
print ("Valor feature:", dump ($features->slice([0,1],[0,0])->asarray));
print ("Valor poli_features:", dump ($poly_features->slice([0,1],[0,19])->asarray));

Valor feature:[[0.00390125624835491], [0.00443110195919871]]Valor poli_features:[
  [
    1,
    0.00390125624835491,
    1.52197999341297e-05,
    5.93763402889635e-08,
    2.31642316439462e-10,
    9.03696063293824e-13,
    3.52554996745677e-15,
    1.37540738482204e-17,
    5.36581658982007e-20,
    2.09334250267561e-22,
    8.16666575800629e-25,
    3.18602546247097e-27,
    1.24295019758773e-29,
    4.84906730528113e-32,
    1.89174532888613e-34,
    7.38018342144209e-37,
    2.87919890920195e-39,
    1.12328084900277e-41,
    4.34402523940693e-44,
    0,
  ],
  [
    1,
    0.00443110195919871,
    1.96346645680023e-05,
    8.70032010880095e-08,
    3.85520060319777e-10,
    1.70827870078771e-12,
    7.5695573669831e-15,
    3.35414790073107e-17,
    1.48625718662602e-19,
    6.58575717827683e-22,
    2.9182159889594e-24,
    1.29309124322576e-26,
    5.72981913121877e-29,
    2.53894131458399e-31,
    1.12503083751897e-33,
    4.98512632695902e-36,
    2.20896023592278e-38,
    

1

## 4.4.4.2. Entrenamiento y prueba del modelo
Primero implementemos una función para evaluar la pérdida en un conjunto de datos dado.

In [135]:
sub train_epoch_ch3{ #@save

  #Train a model within one epoch (defined in Chapter 3).

  #Sum of training loss, sum of training accuracy, no. of examples

  my ($net, $train_iter, $loss, $updater, $batch_size) = @_;

  my $metric = Accumulator->new(3);

  if (ref($updater) eq 'AI::MXNet::Gluon::Trainer'){

    $updater->step($batch_size, 1);

  }  

  my ($X, $y, $y_hat, $l);

  while(defined(my $batch = <$train_iter>)){

    # Compute gradients and update parameters

    $X = $batch->[0];

    $y = $batch->[1]->astype('float32'); 

    autograd->record(sub {

      $y_hat = $net->($X);

      $l = $loss->($y_hat, $y);

    });

    $l->backward();

    if (ref($updater) eq 'AI::MXNet::Gluon::Trainer'){

      $updater->step($batch_size);

    }else{

      $updater->($batch_size);

    }

    $metric->add([ $l->sum->asscalar, accuracy($y_hat, $y), $y->size ]);

  }

  # Return training loss and training accuracy

  return ($metric->getitem(0) / $metric->getitem(2), $metric->getitem(1) / $metric->getitem(2));

}

In [136]:
sub evaluate_loss {
 my ($net,$loss, $data_iter) = @_;
 my $metric = d2l->Accumulator(2);  # Sum of losses, no. of examples
 my ($X, $y, $l);
  
while(defined(my $batch = <$data_iter>)){

    $X = $batch ->[0];
    $y = $batch ->[1]-> astype('float32');
    $l = loss($net->($X), $y);
    $metric->add([$l->sum(), $l->size]);
    }
return ($metric-> getitem(0) / $metric->getitem(1));
}

Warning: Subroutine evaluate_loss redefined at reply input line 1.


Ahora defina la función de entrenamiento.

In [139]:
sub train my ($train_features, $test_features, $train_labels, $test_labels,
          $num_epochs=400)
{
           my $loss = gluon->loss->L2Loss();
    my $net = mx->nd->Sequential();
    # Switch off the bias since we already catered for it in the polynomial
    # features
    my $net->add(nd->Dense(1, my $use_bias=False));
    my $net->initialize();
    my $batch_size = min(10, $train_labels->shape[0]);
    my $train_iter = d2l->load_array(($train_features, $train_labels), $batch_size)
    $test_iter = d2l->load_array(($test_features, $test_labels), $batch_size,
                               is_train=False)
    my $trainer = gluon->Trainer($net->collect_params(), 'sgd',
                            {'learning_rate': 0.01})
    my $animator = d2l->Animator(my $xlabel='epoch', my $ylabel='loss', my $yscale='log',
                            my $xlim=[1, my $num_epochs], my $ylim=[1e-3, 1e2],
                            my $legend=['train', 'test']);
    for epoch in range($num_epochs){
     d2l->train_epoch_ch3($net, $train_iter, $loss, $trainer);
        if my $epoch == 0 or ($epoch + 1) % 20 == 0:
            animator->add(epoch + 1, (evaluate_loss(
                $net, $train_iter, $loss), $evaluate_loss($net, $test_iter, $loss)));
    }
       
          }
    
    print('weight:', net[0]->weight->data()->asnumpy());

Error: Illegal declaration of subroutine main::train at reply input line 1.



## 4.4.4.3. Ajuste de función polinomial de tercer orden (normal)
Comenzaremos usando primero una función polinomial de tercer orden, que es del mismo orden que la función de generación de datos. Los resultados muestran que las pérdidas de entrenamiento y prueba de este modelo pueden reducirse de manera efectiva. Los parámetros del modelo aprendido también están cerca de los valores reales w=[5,1.2,−3.4,5.6] .

In [140]:
#ck the first four dimensions, i.e., 1, x, x^2/2!, x^3/3! from the
# polynomial features
#train(poly_features[:n_train, :4], poly_features[n_train:, :4],
 #     labels[:n_train], labels[n_train:])
      
      
print ("Valor feature:", dump ($features->slice([0,1],[0,4])->asarray));
print ("Valor poli_features:", dump ($poly_features->slice([0,1],[0,4])->asarray));

Error: Bareword "i" not allowed while "strict subs" in use at reply input line 1.
syntax error at reply input line 1, near ".,"
BEGIN not safe after errors--compilation aborted at reply input line 12.



## 4.4.4.4. Ajuste de función lineal (ajuste inferior)
Echemos otro vistazo al ajuste de funciones lineales. Después del declive en las primeras épocas, se vuelve difícil disminuir aún más la pérdida de entrenamiento de este modelo. Una vez completada la última iteración de época, la pérdida de entrenamiento sigue siendo alta. Cuando se utilizan para ajustar patrones no lineales (como aquí la función polinomial de tercer orden), los modelos lineales tienden a no ajustarse correctamente.


train(poly_features[:n_train, :2], poly_features[n_train:, :2],
      labels[:n_train], labels[n_train:]);

## 4.4.4.5. Ajuste de funciones polinómicas de orden superior (sobreajuste)
Ahora intentemos entrenar el modelo usando un polinomio de grado demasiado alto. Aquí, no hay datos suficientes para saber que los coeficientes de mayor grado deberían tener valores cercanos a cero. Como resultado, nuestro modelo excesivamente complejo es tan susceptible que está siendo influenciado por el ruido en los datos de entrenamiento. Aunque la pérdida de entrenamiento se puede reducir de manera efectiva, la pérdida de prueba sigue siendo mucho mayor. Muestra que el modelo complejo sobreajusta los datos.

In [89]:
train(poly_features[:n_train, :], poly_features[n_train:, :],
      labels[:n_train], labels[n_train:], num_epochs=1500);

Error: syntax error at reply input line 1, near "poly_features["
BEGIN not safe after errors--compilation aborted at reply input line 6.



En las secciones subsiguientes, continuaremos analizando los problemas de sobreajuste y los métodos para tratarlos, como la disminución del peso y la deserción.

## 4.4.5. Resumen
Dado que el error de generalización no se puede estimar en función del error de entrenamiento, simplemente minimizar el error de entrenamiento no significará necesariamente una reducción en el error de generalización. Los modelos de aprendizaje automático deben tener cuidado de protegerse contra el sobreajuste para minimizar el error de generalización.

Se puede usar un conjunto de validación para la selección del modelo, siempre que no se use con demasiada liberalidad.

Underfitting significa que un modelo no es capaz de reducir el error de entrenamiento. Cuando el error de entrenamiento es mucho menor que el error de validación, hay sobreajuste.

Deberíamos elegir un modelo adecuadamente complejo y evitar usar muestras de entrenamiento insuficientes.